In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))
    !nvidia-smi

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA A100-SXM4-40GB
Sun May 26 00:19:38 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              48W / 400W |      5MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabl

In [ ]:
!pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.0/289.0 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.0 MB/s eta 0:00:00


In [ ]:
# Log in to your W&B account
import wandb

In [ ]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
 wandb.init(
      # Set the project where this run will be logged
      project="colab-longformer-baseline",
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"experiment_law",
      # Track hyperparameters and run metadata
      config={
      "learning_rate": 0.00005,
      "architecture": "Longformer",
      "dataset": "ehcr-train",
      "epochs": 2,
      'batch':8,
      'data':'conclusion'
      })

wandb: Currently logged in as: ahmedabdou. Use `wandb login --relogin` to force relogin


In [ ]:
!git clone https://github.com/AhmedAbdel-Aal/TND.git
!cp -r TND/baseline/*.py .
!rm -rf TND

Cloning into 'TND'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 51 (delta 10), reused 47 (delta 9), pack-reused 0
Receiving objects: 100% (51/51), 17.03 KiB | 8.51 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [ ]:
!unzip /content/drive/MyDrive/msc-thesis/echr_splits/train.zip -d /content/
!unzip /content/drive/MyDrive/msc-thesis/echr_splits/dev.zip -d /content/
!unzip /content/drive/MyDrive/msc-thesis/echr_splits/test.zip -d /content/

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import AdamW, get_scheduler
from tqdm.auto import tqdm
from model import ECHRModel
import torch.nn as nn
from dataset import ECHRDataset

In [ ]:
from torch.cuda.amp import GradScaler, autocast
# Initialize GradScaler for mixed precision training
scaler = GradScaler()

In [ ]:
echr_train = ECHRDataset(data_path='/content/train')
echr_dev = ECHRDataset(data_path='/content/dev')
echr_test = ECHRDataset(data_path='/content/test')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

Using Tokenizer allenai/longformer-base-4096 with max_length 4096
Using Tokenizer allenai/longformer-base-4096 with max_length 4096
Using Tokenizer allenai/longformer-base-4096 with max_length 4096


In [ ]:
len(echr_train), len(echr_dev), len(echr_test)

(8276, 1216, 6604)

In [ ]:
train_loader = DataLoader(echr_train, batch_size=8, shuffle=True, collate_fn=echr_train.collate_fn, num_workers=5)
dev_loader = DataLoader(echr_dev, batch_size=8, shuffle=True, collate_fn=echr_dev.collate_fn, num_workers=5)
test_loader = DataLoader(echr_test, batch_size=8, shuffle=True, collate_fn=echr_test.collate_fn, num_workers=5)

In [ ]:
# Initialize the model
model = ECHRModel()
model = model.to(device)  # Move the model to GPU if available

# Optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=1e-5)
num_epochs = 20
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

pytorch_model.bin:   0%|          | 0.00/597M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from steps import evaluate

# Training loop
step = 0
best_f1 = 0
model.train()
for epoch in range(num_epochs):
    train_loss = 0.0
    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch + 1}/{num_epochs}"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()

        # Use autocast context manager for mixed precision
        with autocast():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = nn.CrossEntropyLoss()(outputs, labels)

        wandb.log({"train_loss": loss.item()})

        # Scale the loss and perform backpropagation
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        train_loss += loss.item()
        step += 1

    avg_train_loss = train_loss / len(train_loader)
    wandb.log({"epoch": epoch, "avg_train_loss": avg_train_loss})

    # Evaluate on the validation set
    avg_val_loss, f1, micro_f1, macro_f1, accuracy = evaluate(model, dev_loader, device)
    if f1 > best_f1:
      best_f1 = f1
      torch.save(model, '/content/drive/MyDrive/msc-thesis/longformer/law/model_1.pth')

    wandb.log({
        "epoch": epoch,
        "avg_val_loss": avg_val_loss,
        "val_f1_score": f1,
        "val_micor_f1_score": micro_f1,
        "val_macro_f1_score": macro_f1,
        "val_accuracy": accuracy
    })

    print(f"Epoch {epoch + 1}/{num_epochs} - "
          f"Train Loss: {avg_train_loss:.4f} - "
          f"Val Loss: {avg_val_loss:.4f} - "
          f"F1 Score: {f1:.4f} - "
          f"Accuracy: {accuracy:.4f}")

    model.train()


Training Epoch 1/20:   0%|          | 0/1035 [00:00<?, ?it/s]

Validation: 100%|██████████| 152/152 [00:44<00:00,  3.41it/s]


Epoch 1/20 - Train Loss: 0.8458 - Val Loss: 0.7000 - F1 Score: 0.6950 - Accuracy: 0.6982


Training Epoch 2/20:   0%|          | 0/1035 [00:00<?, ?it/s]

Validation: 100%|██████████| 152/152 [01:09<00:00,  2.19it/s]

...

Training Epoch 6/20:   0%|          | 0/1035 [00:00<?, ?it/s]

Validation: 100%|██████████| 152/152 [00:44<00:00,  3.41it/s]

Epoch 6/20 - Train Loss: 0.4858 - Val Loss: 0.7658 - F1 Score: 0.6605 - Accuracy: 0.7155


Training Epoch 7/20:   0%|          | 0/1035 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
model = ECHRModel()
model=torch.load('/content/drive/MyDrive/msc-thesis/longformer/law/model_1.pth')

model = model.to(device)  # Move the model to GPU if available

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
avg_val_loss, f1, micro_f1, macro_f1, accuracy = evaluate(model, dev_loader, device)

print(f"F1 Score: {f1:.4f} - "
      f"Micro-F1 Score: {micro_f1:.4f} - "
      f"Macro-F1 Score: {macro_f1:.4f} - "
      f"Accuracy: {accuracy:.4f}")


Validation: 100%|██████████| 152/152 [00:44<00:00,  3.41it/s]

              precision    recall  f1-score   support

           0       0.80      0.34      0.48        47
           1       0.00      0.00      0.00        32
           2       0.52      0.54      0.53       326
           3       0.81      0.86      0.84       811

    accuracy                           0.73      1216
   macro avg       0.53      0.43      0.46      1216
weighted avg       0.71      0.73      0.72      1216

F1 Score: 0.7178 - Micro-F1 Score: 0.7311 - Macro-F1 Score: 0.4608 - Accuracy: 0.7311


In [ ]:
avg_test_loss, f1, micro_f1, macro_f1, accuracy = evaluate(model, test_loader, device)
print(f"F1 Score: {f1:.4f} - "
      f"Micro-F1 Score: {micro_f1:.4f} - "
      f"Macro-F1 Score: {macro_f1:.4f} - "
      f"Accuracy: {accuracy:.4f}")


Validation: 100%|██████████| 826/826 [03:59<00:00,  3.44it/s]

              precision    recall  f1-score   support

           0       0.75      0.29      0.41       178
           1       0.00      0.00      0.00        35
           2       0.44      0.58      0.50      1135
           3       0.90      0.86      0.88      5256

    accuracy                           0.79      6604
   macro avg       0.52      0.43      0.45      6604
weighted avg       0.81      0.79      0.80      6604

F1 Score: 0.7980 - Micro-F1 Score: 0.7936 - Macro-F1 Score: 0.4484 - Accuracy: 0.7936
